In [2]:
%pip install alpaca-trade-api
%pip install scipy
%pip install mplfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 15.7 MB/s eta 0:00:00
  Created wheel for msgpack: filename=msgpack-1.0.3-cp312-cp312-linux_x86_64.whl size=15688 sha256=0f670784c15e61b51ac4daf72f3ca51d5db395570cd0f826104c51e1dd8d6903
  Stored in directory: /root/.cache/pip/wheels/ba/bd/3f/f043e8f634db9c90ae128d631f43ae9990eef01274a63291f9
  Created wheel for websockets: filename=websockets-10.4-cp312-cp312-linux_x86_64.whl size=107328 sha256=d4e97840a3a2ab2efbdf068d1dc1d3d6215c62076042512ff2f29ad98ef04476
  Stored in directory: /root/.cache/pip/wheels/80/cf/6d/5d7e4c920cb41925a1

In [16]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from alpaca_trade_api.rest import REST, TimeFrame
from scipy import signal
from scipy.signal import find_peaks
from scipy.stats import linregress
import matplotlib.pyplot as plt  # Para visualización opcional
from typing import Optional, Tuple  # Import Optional and Tuple

# ==================== CONFIGURACIÓN ALPACA ====================
# Recomendación: guarda las claves en variables de entorno en vez de hardcodearlas.
API_KEY = os.getenv("ALPACA_API_KEY", "PK6PR8K71SUI4RGI2JMB")
API_SECRET = os.getenv("ALPACA_API_SECRET", "2t6mK57kzfZNFJKVKIAAEI7Grvj0rOrSajwnVXvq")
BASE_URL = "https://paper-api.alpaca.markets"  # Paper trading

def get_alpaca_data(ticker: str, start_date: str, end_date: str, timeframe: str = "1Day") -> pd.DataFrame:
    """
    Descarga datos históricos de Alpaca y los devuelve en formato similar a yfinance.
    """
    alpaca = REST(API_KEY, API_SECRET, base_url=BASE_URL)

    if timeframe == "1Day":
        tf = TimeFrame.Day
    elif timeframe == "1Hour":
        tf = TimeFrame.Hour
    else:
        raise ValueError("Solo se soporta '1Day' o '1Hour' en este ejemplo")

    bars = alpaca.get_bars(ticker, tf, start=start_date, end=end_date).df

    if bars.empty:
        raise ValueError(f"Sin datos de Alpaca para {ticker} entre {start_date} y {end_date}")

    # Normalizar índice y nombres de columnas
    if 'symbol' in bars.columns:
        bars = bars[bars["symbol"] == ticker].set_index("timestamp")
    else:
        if 'timestamp' in bars.columns:
            bars = bars.set_index("timestamp")

    bars = bars.rename(columns={
        "open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"
    })
    bars.index = pd.to_datetime(bars.index)
    bars["Adj Close"] = bars["Close"]
    return bars

# ==================== PARÁMETROS GLOBALES ====================
TICKERS = ["AAPL", "META", "AMZN", "MSFT", "GOOGL", "TSLA","NVDA"]           # <- edita tu canasta
START_DATE = "2023-01-01"
END_DATE   = "2025-11-03"

# ==================== INDICADORES ====================

def EMA(series: pd.Series, period: int) -> pd.Series:
    return series.ewm(span=period, adjust=False).mean()

def calcular_SMA(series: pd.Series, period: int) -> pd.Series:
    return series.rolling(window=period).mean()

def calcular_MACD(close_series: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    ema_fast = EMA(close_series, fast)
    ema_slow = EMA(close_series, slow)
    macd_line = ema_fast - ema_slow
    signal_line = EMA(macd_line, signal)
    histogram = macd_line - signal_line
    macd_slope = macd_line.diff()
    return macd_line, signal_line, histogram, macd_slope

def calcular_RSI_Wilder(close_series: pd.Series, period: int = 14) -> pd.Series:
    delta = close_series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    rsi = 100 - (100 / (1 + rs))
    return rsi

# ==================== MOTOR DE PATRONES ====================

def get_picos_valles(series: pd.Series, order_n: int, prominence: float = 0.01) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    picos_idx = find_peaks(series.values, distance=order_n, prominence=prominence)[0]
    valles_idx = find_peaks(-series.values, distance=order_n, prominence=prominence)[0]
    return picos_idx, series.iloc[picos_idx].values, valles_idx, series.iloc[valles_idx].values

def get_trendline(indices: np.ndarray, values: np.ndarray) -> Optional[Tuple[float, float]]:
    if len(indices) < 2:
        return None
    res = linregress(indices, values)
    return res.slope, res.intercept

def is_near(val1: float, val2: float, tolerance_pct: float = 0.03) -> bool:
    if val2 == 0: return abs(val1) <= tolerance_pct
    return abs(val1 - val2) / abs(val2) <= tolerance_pct

# --- Definiciones de Patrones (Funciones) ---

def detectar_hch(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5) -> str:
    if len(picos_idx) < 3 or len(valles_idx) < 2: return "NEUTRAL"
    P1_idx, P2_idx, P3_idx = picos_idx[-3:]
    P1_val, P2_val, P3_val = picos_val[-3:]
    V1_idx, V2_idx = valles_idx[-2:]
    V1_val, V2_val = valles_val[-2:]
    cond1 = (P2_val > P1_val) and (P2_val > P3_val)
    cond2 = is_near(P1_val, P3_val, tolerance_pct + 0.02)
    cond3 = is_near(V1_val, V2_val, tolerance_pct)
    cond4 = (V1_idx > P1_idx) and (V1_idx < P2_idx)
    cond5 = (V2_idx > P2_idx) and (V2_idx < P3_idx)
    if cond1 and cond2 and cond3 and cond4 and cond5:
        neckline_level = (V1_val + V2_val) / 2
        if current_price < neckline_level:
            avg_vol = data_slice['Volume'].mean()
            recent_vol = data_slice['Volume'].iloc[-1]
            if recent_vol > avg_vol * vol_multiplier:
                return "REVERSION_BAJISTA_HCH"
    return "NEUTRAL"

def detectar_hch_invertido(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5) -> str:
    if len(valles_idx) < 3 or len(picos_idx) < 2: return "NEUTRAL"
    V1_idx, V2_idx, V3_idx = valles_idx[-3:]
    V1_val, V2_val, V3_val = valles_val[-3:]
    P1_idx, P2_idx = picos_idx[-2:]
    P1_val, P2_val = picos_val[-2:]
    cond1 = (V2_val < V1_val) and (V2_val < V3_val)
    cond2 = is_near(V1_val, V3_val, tolerance_pct + 0.02)
    cond3 = is_near(P1_val, P2_val, tolerance_pct)
    cond4 = (P1_idx > V1_idx) and (P1_idx < V2_idx)
    cond5 = (P2_idx > V2_idx) and (P2_idx < V3_idx)
    if cond1 and cond2 and cond3 and cond4 and cond5:
        neckline_level = (P1_val + P2_val) / 2
        if current_price > neckline_level:
            avg_vol = data_slice['Volume'].mean()
            recent_vol = data_slice['Volume'].iloc[-1]
            if recent_vol > avg_vol * vol_multiplier:
                return "REVERSION_ALCISTA_HCH_INVERTIDO"
    return "NEUTRAL"

def detectar_doble_techo(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5) -> str:
    if len(picos_idx) < 2 or len(valles_idx) < 1: return "NEUTRAL"
    P1_val, P2_val = picos_val[-2:]
    P1_idx, P2_idx = picos_idx[-2:]
    V1_val = valles_val[-1]
    V1_idx = valles_idx[-1]
    cond1 = is_near(P1_val, P2_val, tolerance_pct)
    cond2 = (V1_idx > P1_idx) and (V1_idx < P2_idx)
    if cond1 and cond2:
        neckline_level = V1_val
        if current_price < neckline_level:
            avg_vol = data_slice['Volume'].mean()
            recent_vol = data_slice['Volume'].iloc[-1]
            if recent_vol > avg_vol * vol_multiplier:
                return "REVERSION_BAJISTA_DOBLE_TECHO"
    return "NEUTRAL"

def detectar_doble_valle(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5) -> str:
    if len(valles_idx) < 2 or len(picos_idx) < 1: return "NEUTRAL"
    V1_val, V2_val = valles_val[-2:]
    V1_idx, V2_idx = valles_idx[-2:]
    P1_val = picos_val[-1]
    P1_idx = picos_idx[-1]
    cond1 = is_near(V1_val, V2_val, tolerance_pct)
    cond2 = (P1_idx > V1_idx) and (P1_idx < V2_idx)
    if cond1 and cond2:
        neckline_level = P1_val
        if current_price > neckline_level:
            avg_vol = data_slice['Volume'].mean()
            recent_vol = data_slice['Volume'].iloc[-1]
            if recent_vol > avg_vol * vol_multiplier:
                return "REVERSION_ALCISTA_DOBLE_VALLE"
    return "NEUTRAL"

def detectar_triangulo_ascendente(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5, min_points=3) -> str:
    if len(picos_idx) < min_points or len(valles_idx) < min_points: return "NEUTRAL"
    resistencia = get_trendline(picos_idx[-min_points:], picos_val[-min_points:])
    soporte = get_trendline(valles_idx[-min_points:], valles_val[-min_points:])
    if resistencia and soporte:
        res_slope, res_intercept = resistencia
        sup_slope, sup_intercept = soporte
        cond1 = (abs(res_slope) < 0.05)
        cond2 = (sup_slope > 0.1)
        if cond1 and cond2:
            resistencia_level = picos_val[-1]
            if current_price > resistencia_level:
                avg_vol = data_slice['Volume'].mean()
                recent_vol = data_slice['Volume'].iloc[-1]
                if recent_vol > avg_vol * vol_multiplier:
                    return "CONTINUACION_ALCISTA_TRIANGULO_ASC"
    return "NEUTRAL"

def detectar_triangulo_descendente(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5, min_points=3) -> str:
    if len(picos_idx) < min_points or len(valles_idx) < min_points: return "NEUTRAL"
    resistencia = get_trendline(picos_idx[-min_points:], picos_val[-min_points:])
    soporte = get_trendline(valles_idx[-min_points:], valles_val[-min_points:])
    if resistencia and soporte:
        res_slope, res_intercept = resistencia
        sup_slope, sup_intercept = soporte
        cond1 = (res_slope < -0.1)
        cond2 = (abs(sup_slope) < 0.05)
        if cond1 and cond2:
            soporte_level = valles_val[-1]
            if current_price < soporte_level:
                avg_vol = data_slice['Volume'].mean()
                recent_vol = data_slice['Volume'].iloc[-1]
                if recent_vol > avg_vol * vol_multiplier:
                    return "CONTINUACION_BAJISTA_TRIANGULO_DESC"
    return "NEUTRAL"

def detectar_banderas_banderolas(data_slice, current_price, lookback_asta=20, lookback_cons=7, min_move_pct=0.10, vol_multiplier=1.5) -> str:
    if len(data_slice) < lookback_asta: return "NEUTRAL"
    slice_asta = data_slice.iloc[-lookback_asta:]
    min_asta = slice_asta['Low'].min()
    max_asta = slice_asta['High'].max()
    if (max_asta - min_asta) / min_asta > min_move_pct:
        consolidacion = data_slice.iloc[-lookback_cons:]
        trend = get_trendline(np.arange(len(consolidacion)), consolidacion['Close'].values)
        if trend:
            slope, _ = trend
            if abs(slope) < 0.1:
                if current_price > max_asta:
                    avg_vol = data_slice['Volume'].mean()
                    recent_vol = data_slice['Volume'].iloc[-1]
                    if recent_vol > avg_vol * vol_multiplier:
                        return "CONTINUACION_ALCISTA_BANDERA"
    return "NEUTRAL"

# --- Función de Visualización (Opcional) ---
def plot_patrones(data_slice: pd.DataFrame, picos_idx, picos_val, valles_idx, valles_val, detected_pattern: str):
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(data_slice.index, data_slice['Close'], label='Close Price')
    # Asegurar que los índices usados estén dentro del rango del slice:
    if len(picos_idx):
        ax.scatter(data_slice.index[picos_idx], picos_val, marker='^', label='Picos')
    if len(valles_idx):
        ax.scatter(data_slice.index[valles_idx], valles_val, marker='v', label='Valles')
    if 'TRIANGULO' in detected_pattern:
        res_idx = picos_idx[-3:] if len(picos_idx) >= 3 else picos_idx
        res_val = picos_val[-3:] if len(picos_val) >= 3 else picos_val
        sup_idx = valles_idx[-3:] if len(valles_idx) >= 3 else valles_idx
        sup_val = valles_val[-3:] if len(valles_val) >= 3 else valles_val
        res = get_trendline(res_idx, res_val) if len(res_idx) >= 2 else None
        sup = get_trendline(sup_idx, sup_val) if len(sup_idx) >= 2 else None
        if res:
            ax.plot(data_slice.index[res_idx], res[0] * np.array(res_idx) + res[1], label='Resistencia')
        if sup:
            ax.plot(data_slice.index[sup_idx], sup[0] * np.array(sup_idx) + sup[1], label='Soporte')
    ax.set_title(f'Patrón Detectado: {detected_pattern}')
    ax.legend()
    plt.show()

# --- Motor de Patrones (El que llama a las funciones) ---
PATRONES_PRIORIDAD = [
    (detectar_hch, "picos"),
    (detectar_hch_invertido, "picos"),
    (detectar_doble_techo, "picos"),
    (detectar_doble_valle, "picos"),
    (detectar_triangulo_ascendente, "picos"),
    (detectar_triangulo_descendente, "picos"),
    (detectar_banderas_banderolas, "data")
]

def analizar_patrones_recientes(
    data_slice: pd.DataFrame,
    current_price: float,
    order_n: int = 5,
    prominence: float = 0.01,
    tolerance_pct: float = 0.03,
    vol_multiplier: float = 1.5,
    min_points_trend: int = 3,
    lookback_asta: int = 20,
    lookback_cons: int = 7,
    min_move_pct: float = 0.10,
    plot: bool = False
) -> str:
    if data_slice.empty: return "NEUTRAL"
    picos_idx, picos_val, valles_idx, valles_val = get_picos_valles(
        data_slice['Close'], order_n, prominence
    )
    for detector_func, tipo_input in PATRONES_PRIORIDAD:
        kwargs = {'vol_multiplier': vol_multiplier, 'data_slice': data_slice}
        name = detector_func.__name__
        # Ajustes por tipo de detector
        if 'triangulo' in name:
            kwargs.update({'tolerance_pct': tolerance_pct, 'min_points': min_points_trend})
        elif 'doble' in name or 'hch' in name:
            kwargs.update({'tolerance_pct': tolerance_pct})
        elif 'banderas' in name:
            kwargs.update({'lookback_asta': lookback_asta, 'lookback_cons': lookback_cons, 'min_move_pct': min_move_pct})
        # Llamada al detector según tipo_input
        if tipo_input == "data":
            estado = detector_func(data_slice, current_price, **{k:v for k,v in kwargs.items() if k != 'data_slice'})
        else:
            estado = detector_func(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, **{k:v for k,v in kwargs.items() if k != 'data_slice'})
        if estado != "NEUTRAL":
            if plot:
                plot_patrones(data_slice, picos_idx, picos_val, valles_idx, valles_val, estado)
            return estado
    return "NEUTRAL"

# ==================== DETECCIÓN DE ONDAS DE ELLIOTT ====================

def encontrar_pivotes(df, porcentaje_cambio=0.05):
    if 'Close' not in df.columns:
        raise ValueError("El DataFrame debe tener la columna 'Close'.")
    data_y = df['Close'].values
    timestamps = df.index
    peak_indexes = signal.argrelextrema(data_y, np.greater)[0]
    peaks = pd.DataFrame({'idx_num': peak_indexes, 'price': data_y[peak_indexes], 'type': 'high'})
    valley_indexes = signal.argrelextrema(data_y, np.less)[0]
    valleys = pd.DataFrame({'idx_num': valley_indexes, 'price': data_y[valley_indexes], 'type': 'low'})
    all_pivots = pd.concat([peaks, valleys]).sort_values('idx_num').reset_index(drop=True)
    if all_pivots.empty:
        return pd.DataFrame(columns=['timestamp', 'price', 'type'])
    filtered = [all_pivots.iloc[0]]
    previous_price = all_pivots['price'].iloc[0]
    for i in range(1, len(all_pivots)):
        current_price = all_pivots['price'].iloc[i]
        rel_diff = abs(current_price - previous_price) / previous_price if previous_price != 0 else 0
        if rel_diff >= porcentaje_cambio:
            filtered.append(all_pivots.iloc[i])
            previous_price = current_price
    if not filtered:
         return pd.DataFrame(columns=['timestamp', 'price', 'type'])
    filtered_pivots = pd.concat(filtered, axis=1).T
    filtered_pivots['timestamp'] = timestamps[filtered_pivots['idx_num'].values.astype(int)]
    # Normalizar tipos: usar 'low'/'high' como en el validador
    filtered_pivots['type'] = filtered_pivots['type'].replace({'high':'high','low':'low'})
    return filtered_pivots[['timestamp', 'price', 'type']]

def validar_impulso_elliott(pivotes, usar_fibonacci=True):
    if len(pivotes) < 6:
        return False, None
    for i in range(len(pivotes) - 5):
        seq = pivotes.iloc[i:i+6]
        expected_types = ['low', 'high', 'low', 'high', 'low', 'high']
        if seq['type'].tolist() != expected_types:
            continue
        P0, P1, P2, P3, P4, P5 = seq['price'].values
        if P2 <= P0:
            continue
        wave1 = P1 - P0
        wave3 = P3 - P2
        wave5 = P5 - P4
        if wave3 < wave1 and wave3 < wave5:
            continue
        if P4 <= P1:
            continue
        if usar_fibonacci and wave3 < 1.618 * wave1:
            continue
        return True, seq
    return False, None

# ==================== INTERPRETACIÓN DE SEÑALES ====================

def interpretar_rsi(rsi_value: float) -> str:
    if rsi_value > 70:
        return "sell"
    elif rsi_value < 30:
        return "buy"
    else:
        return "neutral"

def interpretar_macd(macd_value: float, signal_value: float) -> str:
    if macd_value > signal_value:
        return "buy"
    elif macd_value < signal_value:
        return "sell"
    else:
        return "neutral"

def interpretar_ma(close: float, sma50: float, sma200: float) -> str:
    if sma50 is None or sma200 is None or sma50 == 0 or sma200 == 0:
        return "neutral"
    if close > sma50 and sma50 > sma200:
        return "buy"
    elif close < sma50 and sma50 < sma200:
        return "sell"
    else:
        return "neutral"

def interpretar_cross(sma50_series: pd.Series, sma200_series: pd.Series) -> str:
    if sma50_series is None or sma200_series is None:
        return "neutral"
    if len(sma50_series.dropna()) < 2 or len(sma200_series.dropna()) < 2:
        return "neutral"
    if sma50_series.iloc[-2] < sma200_series.iloc[-2] and sma50_series.iloc[-1] > sma200_series.iloc[-1]:
        return "buy"
    elif sma50_series.iloc[-2] > sma200_series.iloc[-2] and sma50_series.iloc[-1] < sma200_series.iloc[-1]:
        return "sell"
    else:
        return "neutral"

# ==================== LÓGICA DE DECISIÓN ====================

def get_decision(patron: str, es_elliott_valido: bool, cross_sig: str, ma_sig: str, macd_last: float, signal_last: float, rsi_last: float) -> str:
    patron_sig = "neutral"
    if patron != "NEUTRAL":
        if "ALCISTA" in patron:
            patron_sig = "buy"
        elif "BAJISTA" in patron:
            patron_sig = "sell"
    if es_elliott_valido:
        patron_sig = "buy"
    if cross_sig != "neutral":
        patron_sig = cross_sig
    if patron_sig != "neutral":
        return patron_sig if patron_sig in ["buy", "sell"] else "hold"
    trend = ma_sig
    if trend == "neutral":
        return "hold"
    macd_above = macd_last > signal_last
    macd_below = macd_last < signal_last
    rsi_over = rsi_last > 70
    rsi_under = rsi_last < 30
    if trend == "buy":
        if macd_above and rsi_over:
            return "hold"
        elif not macd_above and rsi_over:
            return "sell"
        elif macd_above and not rsi_over:
            return "buy"
        else:
            return "hold"
    elif trend == "sell":
        if macd_below and rsi_under:
            return "hold"
        elif not macd_below and rsi_under:
            return "buy"
        elif macd_below and not rsi_under:
            return "sell"
        else:
            return "hold"
    return "hold"

# ==================== EJECUCIÓN ====================

PERIODS = [5, 10, 20, 50, 100, 200]

print("\n\n========================= RESUMEN DE INDICADORES CON INTERPRETACIÓN =========================\n")

for ticker in TICKERS:
    try:
        print(f"Ticker: {ticker}")
        df = get_alpaca_data(ticker, START_DATE, END_DATE)
        close_last = df['Close'].iloc[-1]
        smas = {p: calcular_SMA(df['Close'], p).iloc[-1] for p in PERIODS if len(df) >= p}
        print("  SMA:")
        for p, val in smas.items():
            print(f"    {p}: {val:.2f}")
        emas = {p: EMA(df['Close'], p).iloc[-1] for p in PERIODS}
        print("  EMA:")
        for p, val in emas.items():
            print(f"    {p}: {val:.2f}")
        sma50_series = calcular_SMA(df['Close'], 50)
        sma200_series = calcular_SMA(df['Close'], 200)
        macd_line, signal_line, histogram, macd_slope = calcular_MACD(df['Close'])
        rsi = calcular_RSI_Wilder(df['Close'])
        rsi_last = rsi.iloc[-1]
        macd_last = macd_line.iloc[-1]
        signal_last = signal_line.iloc[-1]
        rsi_sig = interpretar_rsi(rsi_last)
        macd_sig = interpretar_macd(macd_last, signal_last)
        ma_sig = interpretar_ma(close_last, smas.get(50, None), smas.get(200, None))
        cross_sig = interpretar_cross(sma50_series, sma200_series)
        pivotes = encontrar_pivotes(df, porcentaje_cambio=0.05)
        es_valido, patron_elliott = validar_impulso_elliott(pivotes)
        elliott_sig = "buy" if es_valido else "neutral"
        patron = analizar_patrones_recientes(df, close_last, order_n=5, plot=False)
        print(f"  RSI (último): {rsi_last:.2f} → {rsi_sig}")
        print(f"  MACD ahora: {macd_last:.4f}  Señal: {signal_last:.4f} → {macd_sig}")
        print(f"  MA (basado en SMA50 y SMA200): {ma_sig}")
        print(f"  Cross (Golden/Death): {cross_sig}")
        print(f"  Elliott Impulse: {elliott_sig}")
        print(f"  Patrón Detectado: {patron}")
        decision = get_decision(patron, es_valido, cross_sig, ma_sig, macd_last, signal_last, rsi_last)
        print(f"  Decisión Final (buy/hold/sell): {decision}")
        print("======================================================================\n")
    except Exception as e:
        print(f"Error procesando {ticker}: {e}")




========================= RESUMEN DE INDICADORES CON INTERPRETACIÓN =========================

Ticker: AAPL
  SMA:
    5: 269.90
    10: 266.19
    20: 259.13
    50: 249.07
    100: 230.66
    200: 223.51
  EMA:
    5: 268.92
    10: 266.06
    20: 261.23
    50: 249.97
    100: 238.28
    200: 228.97
  RSI (último): 66.46 → neutral
  MACD ahora: 6.2144  Señal: 5.4850 → buy
  MA (basado en SMA50 y SMA200): buy
  Cross (Golden/Death): neutral
  Elliott Impulse: neutral
  Patrón Detectado: NEUTRAL
  Decisión Final (buy/hold/sell): buy

Ticker: META
  SMA:
    5: 691.13
    10: 714.55
    20: 715.91
    50: 736.26
    100: 732.94
    200: 678.67
  EMA:
    5: 676.38
    10: 698.22
    20: 713.07
    50: 725.93
    100: 716.50
    200: 680.07
  RSI (último): 25.43 → buy
  MACD ahora: -15.2874  Señal: -6.7395 → sell
  MA (basado en SMA50 y SMA200): neutral
  Cross (Golden/Death): neutral
  Elliott Impulse: neutral
  Patrón Detectado: REVERSION_BAJISTA_DOBLE_TECHO
  Decisión Final (buy/ho